# Models, Prompts and Output Parsers

## Install dependencies

In [1]:
%%capture
%pip install langchain langchain-community langchain-ollama langchain-chroma

### Model

Use Ollama to get a local LLM

In [2]:
from langchain_ollama import ChatOllama

chat = ChatOllama(model='llama3.2:1b', temperature=0.0)
chat

ChatOllama(model='llama3.2:1b', temperature=0.0)

### Prompt template

Prompts helps on standardizing or adding more context to the LLM beyond user input.

In [3]:
from langchain.prompts import ChatPromptTemplate

template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

prompt_template = ChatPromptTemplate.from_template(template_string)

print(f"Prompt template: {prompt_template.messages[0].prompt}")
print(f"Prompt input variables: {prompt_template.messages[0].prompt.input_variables}")


Prompt template: input_variables=['style', 'text'] input_types={} partial_variables={} template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n'
Prompt input variables: ['style', 'text']


Create customer message prompt template

In [4]:
customer_style = """American English \
in a calm and respectful tone
"""

customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

customer_messages = prompt_template.format_messages(style=customer_style, text=customer_email)

print(f"Result type: {type(customer_messages)}")
print(f"Message Type: {type(customer_messages[0])}")
print(f"Message: {customer_messages[0]}")

Result type: <class 'list'>
Message Type: <class 'langchain_core.messages.human.HumanMessage'>
Message: content="Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!\n```\n" additional_kwargs={} response_metadata={}


Call the LLM to translate to the style of the customer message

In [5]:
customer_response = chat.invoke(customer_messages)
print(f"Response: {customer_response.content}")

Response: Here's a rewritten version of the text in an American English style, in a calm and respectful tone:

"I'm really upset about my blender lid flying off and leaving a mess on my kitchen walls with smoothies. To make things worse, I don't have the insurance coverage to cover the cost of cleaning up this mess. That's why I was wondering if you could help me out right away."


Create prompt template for service reply (reuse customer template)

In [6]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

service_messages = prompt_template.format_messages(style=service_style_pirate, text=service_reply)
service_response = chat.invoke(service_messages)

print(f"Response: {service_response.content}")

Response: Arrr, me hearty customer be wantin' a translation o' that scurvy text, eh? Alright then, let's set sail fer a polite tone, savvy?

```text
"Hey there, valued customer, we regret to inform ye that our warranty doesn't cover the costs associated with cleanin' yer kitchen. It seems ye've been havin' a bit o' trouble keepin' yer blender in one piece, and for that, we can only offer our sincerest apologies. We hope ye'll take this as a reminder to always follow proper safety protocols when usin' yer appliances. Fair winds and following seas!"
```


## Output Parsers

Let's start with defining how we would like the LLM output to look like:

{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

Even by using the chat prompt template and specify about returning or extract a json file, the response will be a string. 

In [7]:
from langchain.prompts import ChatPromptTemplate

customer_review = """\
This leaf blower is pretty amazing. It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

prompt_template = ChatPromptTemplate.from_template(review_template)
messages = prompt_template.format_messages(text=customer_review)
response = chat.invoke(messages)

print(f"Response Type: {type(response.content)}")
print(f"Response: {response.content}")

Response Type: <class 'str'>
Response: Here is the extracted information in the required format:

```json
{
    "gift": true,
    "delivery_days": 2,
    "price_value": ["It arrived in two days", "I've been using it every other morning to clear the leaves on our lawn"]
}
```

Note: The price value sentences are not directly related to the gift or delivery information, but rather a separate observation about the product. If you'd like to include them in the output as well, I can modify the code accordingly.


For this reason, it is necessary to use output parsers.

### Parse the LLM output string into a Python dictionary

In [8]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

gift_schema = ResponseSchema(
  name="gift",
  description="Was the item purchased\
    as a gift for someone else? \
    Answer True if yes,\
    False if not or unknown."
)

delivery_days_schema = ResponseSchema(
  name="delivery_days",
  description="How many days\
    did it take for the product\
    to arrive? If this \
    information is not found,\
    output -1."
)

price_value_schema = ResponseSchema(
  name="price_value",
  description="Extract any\
    sentences about the value or \
    price, and output them as a \
    comma separated Python list."
)

output_parser = StructuredOutputParser.from_response_schemas([gift_schema, delivery_days_schema, price_value_schema])

format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased    as a gift for someone else?     Answer True if yes,    False if not or unknown.
	"delivery_days": string  // How many days    did it take for the product    to arrive? If this     information is not found,    output -1.
	"price_value": string  // Extract any    sentences about the value or     price, and output them as a     comma separated Python list.
}
```


In [9]:
review_template_with_instructions = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_with_instructions)

messages = prompt.format_messages(text=customer_review, format_instructions=format_instructions)
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing. It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```"

In [10]:
response = chat.invoke(messages)
print(response.content)

```json
{
    "gift": true,
    "delivery_days": "2",
    "price_value": "slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}
```


In [11]:
output_dict = output_parser.parse(response.content)
print(f"Output Dict Type: {type(output_dict)}")
print(f"Output Dict: {output_dict}")
print(f"Delivery Days: {output_dict['delivery_days']}")

Output Dict Type: <class 'dict'>
Output Dict: {'gift': True, 'delivery_days': '2', 'price_value': "slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."}
Delivery Days: 2
